# Purpose

This file shows the steps we took to process the raw data files, zipped xml files.

### Connect with Google drive to access data

In order to access the data, you first need to create a shortcut of the data folder to your own Gdrive. If you've been granted editing rights, you should be able to edit the content of the folder, i.e. add, move and delete data, create and rename folders, etc.

In [4]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# redirect the working directory of this script to the data folder
#%cd /content/drive/MyDrive/data/
%cd /content/drive/MyDrive/Work/Frontline/data

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [6]:
import os
# check number of files 
num_files = len(os.listdir("Raw"))
print("Number of files in the folder: ", num_files)

Number of files in the folder:  257


## Unzip the files 

The first step was to unzip the files and to move the unzipped xml files to another folder called "unzipped". Next, we check whether any zip file in the "raw" data folder is missing a corresponding xml file in the "unzipped" folder. 

In [10]:
os.listdir("./Raw")

['NBPC_part1.zip', 'NBPC_part2.zip', 'NBPC_part4.zip', 'NBPC_part3.zip']

In [11]:
# unzip all files 

import zipfile
import shutil

# Path to the raw folder
raw_folder = "./Raw"

# Path to the unzipped folder
unzipped_folder = "./unzipped"

# Loop through all the files in the raw folder
for filename in os.listdir("./Raw")[-4:]:#os.listdir(raw_folder):
    if filename.endswith(".zip"):
        # Check if the file is a valid zip file
        if zipfile.is_zipfile(os.path.join(raw_folder, filename)):
            # Create a ZipFile object for the current zip file
            with zipfile.ZipFile(os.path.join(raw_folder, filename), "r") as zip_ref:
                # Extract all the contents of the zip file to the unzipped folder
                zip_ref.extractall(unzipped_folder)
        else:
            print(f"File {filename} is not a valid zip file and will not be extracted.")

In [ ]:
# check whether a zip file is missing a xml file 

# Loop through all the files in the Raw folder
for filename in os.listdir(raw_folder):
    if filename.endswith(".zip"):
        # Get the first part of the filename before the .zip extension
        name = filename.split(".")[0]

        # Loop through all the files in the unzipped folder
        for xml_filename in os.listdir(unzipped_folder):
            if xml_filename.startswith(name) and xml_filename.endswith(".xml"):
                break
        else:
            # A corresponding XML file doesn't exist
            print(f"No corresponding XML file exists for {filename}")

## Reading XMLS and filtering
In this step all prefixes are looped through. Before parsing the xmls files, it is checked if a csv file for that prefix already exists. If not, all the XMls with that prefix are read and filtered. If they contain words/ phrases associated with domestic violence, they are exported as csv, using their prefix as name. 

#### Methods
For this step, the following methods are defined:
* parsing xml files
* testing if csv was exported already
* checking if text contains DV relatedwords


In [12]:
def parse_xml_file(xml_file):
  """ function to combine all xml files of a prefix and returns it as json
  Parameters:
    - prefix (str): prefix of the journal 
  Returns:
    - json: containing the combined xml files of a prefix in json format
  """
  tree = ET.parse(os.path.join(input_dir, xml_file))
  # Get the root 
  root = tree.getroot()
  # Create list for converted output 
  json_file = []
  # Loop through each article, get the data and append it to the output file json_file
  for artikel in root.findall('artikel'):
    # Access static data by their xpath
    # Store data unless data is not available and None, then store as None 
    artikel_id = artikel.find('metadaten/artikel-id')
    artikel_id = artikel_id.text if artikel_id is not None else None 

    name = artikel.find('metadaten/quelle/name').text

    jahrgang = artikel.find('metadaten/quelle/jahrgang')
    jahrgang = jahrgang.text if jahrgang is not None else None

    datum = artikel.find('metadaten/quelle/datum')
    datum = datum.text if datum is not None else None


    # Access variable data by their xpath 
    ressort_elem = artikel.find('inhalt/titel-liste/ressort')
    # Store data unless data is not available and None, then store as None 
    ressort = ressort_elem.text if ressort_elem is not None else None 

    titel_elem = artikel.find('inhalt/titel-liste/titel')
    titel = titel_elem.text if titel_elem is not None else None 

    untertitel_elem = artikel.find('inhalt/titel-liste/untertitel')
    untertitel = untertitel_elem.text if untertitel_elem is not None else None

    # Create list for text inputs 
    text = []
    # Find the 'text' element
    text_elem = artikel.find('inhalt/text')
    try: 
        # Extract all the 'p' elements inside the 'text' element
        p_elems = text_elem.findall('p')
        # Loop over the 'p' elements and extract their text content
        for p_elem in p_elems:
            p_text = p_elem.text
            # Only add text if text is not empty 
            if p_text is not None: 
              text.append(p_text)

    # If no text element exists, pass 
    except: 
        pass 

    # Create temporary dict to store all information 
    temp_dict = {}
    temp_dict['artikel_id'] = str(artikel_id)
    temp_dict['name'] = name
    temp_dict['jahrgang'] = jahrgang
    temp_dict['datum'] = datum
    temp_dict['ressort'] = ressort
    temp_dict['titel'] = titel
    temp_dict['untertitel'] = untertitel
    temp_dict['text'] = text

    # Add the article dict to the output list 
    json_file.append(temp_dict)
  return json_file


In [13]:
def filter_strings(text, combination_list):
  """ function to compare if any of a list of words occur in a text or or list of texts 
  Parameters:
    - text (str or list of str): text which is checked 
    - combination_list (list of str): list of words which are checked if they occur in text 
  Returns:
    - boolean: True or False depending on if any of the words in combination_list occurs in text
  """

  text="".join(text).lower()
  combinations = combination_list
  for comb in combinations:
    
    if all(word in text for word in comb.split(" und ")):
        return True
  return False

In [14]:
comb1 = ['häusliche gewalt',
        'partnerschaftsgewalt',
        'partnergewalt',
        'femizid',
        'beziehungstat',
        'liebesdrama',
        'ehedrama',
        'liebestragödie',
        'eheliche gewalt',
        'ehekrieg',
        'innerfamiliäre gewalt',
        'innerhäusliche gewalt',
        'gewalt und ehe',
        'ehe und hölle'
        'gewalt und (freund(in)?|partner(in)?|mann|frau)\s',
        'vergewaltigen und (freund(in)?|partner(in)?|mann|frau)\s',
        'vergewaltigung und (freund(in)?|partner(in)?|mann|frau)\s',
        'missbrauch und (freund(in)?|partner(in)?|mann|frau)\s',
        'missbräuchlich und (freund(in)?|partner(in)?|mann|frau)\s',
        'gewalttätig und (freund(in)?|partner(in)?|mann|frau)\s',
        'verletzung und (freund(in)?|partner(in)?|mann|frau)\s',
        'verletzen und (freund(in)?|partner(in)?|mann|frau)\s',
        'übergriffig und (freund(in)?|partner(in)?|mann|frau)\s',
        'drohung und (freund(in)?|partner(in)?|mann|frau)\s',
        'drohen und (freund(in)?|partner(in)?|mann|frau)\s',
        'manipulation und (freund(in)?|partner(in)?|mann|frau)\s',
        'manipulieren und (freund(in)?|partner(in)?|mann|frau)\s',
        'beleidigen und (freund(in)?|partner(in)?|mann|frau)\s',
        'beleidigung und (freund(in)?|partner(in)?|mann|frau)\s',
        'gaslighting und (freund(in)?|partner(in)?|mann|frau)\s',
        'schlagen und (freund(in)?|partner(in)?|mann|frau)\s',
        'zwingen und (freund(in)?|partner(in)?|mann|frau)\s', 
        'gezwungen und (freund(in)?|partner(in)?|mann|frau)\s',
        'zwang und (freund(in)?|partner(in)?|mann|frau)\s',
        'einsperren und (freund(in)?|partner(in)?|mann|frau)\s',
        'stalking und (freund(in)?|partner(in)?|mann|frau)\s',
        'stalken und (freund(in)?|partner(in)?|mann|frau)\s',
        'kontrollieren und (freund(in)?|partner(in)?|mann|frau)\s',
        'kontrolle und (freund(in)?|partner(in)?|mann|frau)\s',
        'isolieren und (freund(in)?|partner(in)?|mann|frau)\s',
        'isolation und (freund(in)?|partner(in)?|mann|frau)\s',
        'hauen und (freund(in)?|partner(in)?|mann|frau)\s',
        'ohrfeige und (freund(in)?|partner(in)?|mann|frau)\s'

        ]

### Filtering and Exporting

In [15]:
# load packages and set directories 
import os 
import pandas as pd
import xml.etree.ElementTree as ET 
from tqdm import tqdm # for progress bar 
import json
import re

# Set up paths for input and output directories 
input_dir = "unzipped"
output_dir = "json"


In [24]:
def check_if_exported(prefix):
  """ function to check if the relevant articles of a journal have been exported already
  Parameters:
    - prefix (str): prefix of the journal that is checked
  Returns:
    - boolean: returns True if the relevant articles of a journal have been exported already, False otherwise
  """
  filtered=os.listdir("filtered2")
  filtered = [file.split(".")[0]for file in filtered if file.endswith(".csv")]
  if prefix in filtered:
    print(f"{prefix} csv already exported")
    return True
  else: 
    return False

In [21]:
# list with all prefixes
prefixes= sorted([i.split(".")[0] for i in os.listdir("Raw")])

['NBPC_part1', 'NBPC_part2', 'NBPC_part4', 'NBPC_part3']

In [ ]:
# looping through all prefixes
for prefix in tqdm(prefixes):
  if not check_if_exported(prefix):
    # list all xmls of a prefix
    xmls=[i for i in os.listdir(input_dir) if i.startswith(prefix+"_")]
    # create an empty list for the json files
    comb_xml=[]
    # loop through all xmlfile
    for xml in xmls:
      # parse each xml
      comb_xml=comb_xml+parse_xml_file(xml)
    #loop through all articles  
    DV_art=[]
    for row_j in range(len(comb_xml)):
      #check if articles are associated with DV
      if filter_strings(comb_xml[row_j]["text"],comb1):
          # DV articles are added to a dictionary
          DV_art.append(comb_xml[row_j])  
    pd.DataFrame(DV_art).to_csv("filtered2/"+prefix+".csv")

## to do list:

* **do filtering: german newspapers only, DA related topics only**

* do descriptive analysis: number of articles per newspaper, number of newspapers, number of topics per newspaper, etc.

* do a collocation analysis: see gitub repo "newspaper" under scripts

* topic analysis: run the filtered dataset through a generic topic model
